# Paraphronima corneas

## Inference

First open your heatmap in napari. 

In [ ]:
# import modules
import torchio as tio

# load in the nifti file with torchio
image_path = '/media/jake/Dropbox_mirror/Smithsonian Dropbox/Jan Hemmi/hyperiid scans/Scans_nifti/Paraphronima_crassipes_f536_u1701837_head/Paraphronima_crassipes_f536_u1701837_head.nii'

image = tio.ScalarImage(image_path)



In [ ]:
image

In [ ]:
# view file with napari
import napari


viewer = napari.view_image(image.numpy(), name='image')


In napari, label a few of the features (that are next to each other) to estimate the resample ratio, then save it as a csv file and load it in the next step


In [ ]:
import pandas as pd
# 2nd scan
#RR = pd.read_csv('/home/jake/projects/dhr/FEG20190213_01_corneas_resample_ratio.csv')
# 3rd scan
# RR = pd.read_csv('/home/jake/projects/dhr/Cystisoma_sp_FEG221129_366_corneas_estimating_resample_ratio.csv')
RR = pd.read_csv('resample_ratio_measurements/resample_ratio_measurement_Paraphronima_crassipes_f536_u1701837_head.csv')

RR

In [ ]:
from scipy.spatial import distance_matrix

coordinates = RR[['axis-1', 'axis-2', 'axis-3']].values
dist_matrix = distance_matrix(coordinates, coordinates) #Create a distance matrix


In [ ]:
import numpy as np
np.fill_diagonal(dist_matrix, np.inf)

# Find the nearest neighbor distance for each point
nearest_distances = np.min(dist_matrix, axis=1)

# Add the nearest distances to the dataframe
RR['nearest_distance'] = nearest_distances
RR

In [ ]:
mean_nearest_distance = RR['nearest_distance'].mean()
mean_nearest_distance

In [ ]:
resample_ratio = mean_nearest_distance/10 #Makes average distance between features 10 voxels
resample_ratio

## Inference

In [ ]:
!python main.py infer -h #If you need help for parameters in infer

Make inference first with 1 direction in the x, y, and z axis before creating inference in 3 directions. This will produce three files: two csv files (one for the resampled space, and the other not), and a nifty file with predictions.

*Note: these files are currently saved to ./dhr/output/

In [ ]:
#Inference with 1 direction in x, y, and z
# !python main.py infer configs/paraphronima_corneas.yaml -v '/media/jake/Dropbox_mirror/Smithsonian Dropbox/Jan Hemmi/hyperiid scans/Scans_nifti/Paraphronima_crassipes_f536_u1701837_head/Paraphronima_crassipes_f536_u1701837_head.nii' -m ./logs/paraphronima_corneas/lightning_logs/version_13/ -rr 3.1768202046528278 -nx 1 -ny 1 -nz 1

# Inference in 3 directions with 0.25 threshold
!python main.py infer configs/paraphronima_corneas.yaml -v '/media/jake/Dropbox_mirror/Smithsonian Dropbox/Jan Hemmi/hyperiid scans/Scans_nifti/Paraphronima_crassipes_f536_u1701837_head/Paraphronima_crassipes_f536_u1701837_head.nii' -m ./logs/paraphronima_corneas/lightning_logs/version_13/ -rr 3.1768202046528278 -nx 3 -ny 3 -nz 3 --average_threshold 0.25 -ipmv 0.25

You can then view your inference in napari. It might be useful to you, especially if you are trying many inferences with modified parameters, to evalute the performance of each and track estimated % of correct and incorrect inference points in an Excel spreadsheet for example.

In [ ]:
import napari
import numpy as np
import torchio as tio


image = tio.ScalarImage(image_path)

heatmap_path = './output/Paraphronima_crassipes_f536_u1701837_head.logs_paraphronima_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.25_prediction.nii'
#Make sure to use resampled space csv here.
y_hat_path = 'output/Paraphronima_crassipes_f536_u1701837_head.logs_paraphronima_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.25_prediction_peak_min_val_0_25_method_center_of_mass.resampled_space_peaks.csv'

heatmap = tio.ScalarImage(heatmap_path)


viewer = napari.view_image(heatmap.numpy(), name='heatmap', contrast_limits=[0, 1])
#viewer.add_image(image.numpy(), name='image')

# load the coordinates to analyse
def load_coordinates(path, flip_axes=False, mct_path=None):
    locations = np.loadtxt(
        path,
        delimiter=',',
        ndmin=2,
        dtype=np.float64
    )
    if not flip_axes: 
        return locations.tolist()

    if not mct_path:
        raise Exception('You must specify `mct_path` if you need to flip_axes')

    mct = tio.ScalarImage(mct_path)
    locations[:,0] = mct.shape[1] - locations[:,0]
    locations[:,1] = mct.shape[2] - locations[:,1]

    return locations.tolist()


y_hat = load_coordinates(y_hat_path)


viewer.add_points(y_hat)
